In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Reasoning Engine in Vertex AI - Chat history management
@forusone (shins777@gmail.com)

This colab explains how to deal with chat history being stored in firestore.   

Reference : https://python.langchain.com/v0.2/docs/integrations/memory/google_firestore/


## Init Vertex AI SDK

In [1]:
!pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    "google-cloud-firestore" \
    "langchain-google-firestore" \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.4/109.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.0/409.0 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00


In [2]:
import sys
from IPython.display import display, Markdown

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Configuration and initialization

In [3]:
# Static configuration
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://reasoning_7424"
MODEL = "gemini-1.5-flash"

# Firestore
DATABASE="chat-history"
COLLECTION="history"


### Initialize Vertex AI

In [4]:
import vertexai
from vertexai.preview import reasoning_engines

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

## Firestore setting for chat history

Before executing the following cell, you have to set up firestore database in GCP Console.
* Create database : https://cloud.google.com/firestore/docs/manage-databases

### Helper method to get session history

In [5]:
# refer to : https://python.langchain.com/v0.2/docs/integrations/memory/google_firestore/#custom-client

def get_session_history(session_id: str):
    """
    Retrieve chat history data with session id for each conversation from Firestore on GCP
    """

    from langchain_google_firestore import FirestoreChatMessageHistory
    from google.cloud import firestore

    client = firestore.Client(project=PROJECT_ID, database=DATABASE)

    chat_history = FirestoreChatMessageHistory(
                      client=client,
                      session_id=session_id,
                      collection=COLLECTION,
                      encode_message=False,
    )

    return chat_history

def get_converstion(chat_history):
  """
  Get conversation from chat history
  """

  from langchain.schema import HumanMessage, AIMessage
  import langchain_core.messages

  for message in chat_history.messages:
      if type(message) is HumanMessage:
        print(f"Human : {message.content}")
      elif type(message) is AIMessage:
        print(f"AI : {message.content}")


def interactive_chat(query, session_id):
  """
  Chat with agent with chat history.
  """
  respone = agent.query(input=query,
                        config={"configurable": {"session_id": session_id}},
  )
  return respone['output']


### Unit test of Firestore

In [6]:
session_id = "conv-004"

chat_history = get_session_history(session_id)
chat_history.clear()
print(chat_history.messages)

[]


## Integrate reasoning engine and chat history

### Tools definition

In [7]:
# Define Google Search tool
from vertexai.generative_models import grounding, Tool
grounded_search_tool = Tool.from_google_search_retrieval(
    grounding.GoogleSearchRetrieval()
)

### Agent - Local unit test

In [8]:
from vertexai.preview.generative_models import ToolConfig

# Agent Define
agent = reasoning_engines.LangchainAgent(
    model=MODEL,
    tools=[grounded_search_tool,],
    chat_history=get_session_history,  # function to get FirestoreChatMessageHistory
    agent_executor_kwargs={"return_intermediate_steps": False},
)

### Add chat come from interaction into chat history


In [12]:
from time import perf_counter

while True:
  query = input('사용자: ')

  if query == '종료': break

  t1_start = perf_counter()
  output = interactive_chat(query, session_id)
  t1_end = perf_counter()

  display(Markdown(f"{t1_end-t1_start}"))
  display(Markdown(f"AI Agent : {output}"))
  print(f"------------------------------------ ")

chat_history = get_session_history(session_id)
chat_history.clear()
print(chat_history.messages)

사용자: 안녕하세요.


2.4415826069999866

AI Agent : "안녕하세요"는 한국어에서 가장 흔한 인사말입니다.  "안녕"은 "편안하다"라는 뜻으로, "안녕하세요"는 "잘 지내시냐?"는 의미와 같습니다.  즉, 상대방의 안녕을 묻는 인사입니다.  

"안녕하세요"는 "안녕하십니까"와 비슷한 의미를 가지고 있지만, "안녕하십니까"가 더 공식적인 인사말입니다.  

"안녕하세요"는 "잘 지내세요?"와 같은 뜻이지만, "잘 지내세요?"는 더 친근한 인사말입니다. 

"안녕하세요"는 영어로 "Hello" 또는 "Good morning/afternoon/evening"과 같은 뜻입니다. 


------------------------------------ 
사용자: 다우기술의 최신 정보 알려주세요.


3.501089724000053

AI Agent : 다우기술은 IT 서비스 전문 기업으로, IBM, Vmware, Citrix, RedHat 등과의 총판 계약을 통해 해외 유명 솔루션을 국내에 도입하는 사업을 하고 있습니다. 2008년 유니텔네트웍스와 테라스테크놀로지를 합병하여 인터넷 서비스 사업과 메일 솔루션 사업으로 사업 영역을 확대하였습니다.  현재는 보안 사업, PG 사업, 전자 복권 사업, SaaS형 스마트 프로세스 사업, 공공 SI 및 IDC 사업 등을 영위하고 있습니다.

다우기술은 2024년 8월 기준 매출 감소에도 불구하고 수익성이 상승한 것으로 나타났습니다.  이는 시스템 구축 및 건물 관리 부문의 성장과 키움증권의 리테일 및 IB 부문 성장 때문입니다.  하지만 홀세일 부문의 부진과 투자 운용 부문의 역성장으로 매출은 감소했습니다.  인건비 증가 등으로 판관비 부담이 확대되었지만, 원가 구조 개선으로 전년 동기 대비 영업이익률이 상승했고, 순이익률도 상승했습니다.

다우기술은 쇼핑몰 통합 솔루션인 사방넷의 연동 제휴, 클라우드 관련 시스템 구축 수주 등 자체 사업의 성장과 키움증권 리테일 부문의 견조한 성장으로 매출 회복 가능성이 높다고 예상됩니다.  

최신 뉴스에 따르면, 다우기술의 메시징 브랜드 뿌리오는 국내 최초로 발송 목적에 최적화된 메시지를 자동으로 생성 및 제안해 주는 'AI문자 맞춤 제안 서비스'를 출시했습니다.  

다우기술의 현재 주가는 17,960원입니다. 


------------------------------------ 
사용자: 종료
[]


In [16]:
session_id = "conv-003"
chat_history = get_session_history(session_id)
chat_history.messages

[HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}),
 AIMessage(content='"안녕하세요"는 한국어에서 가장 일반적인 인사말입니다.  "안녕하세요"는 "무탈하시오?"와 같이 "잘 지내시나요?"라고 해석할 수 있습니다.  "안녕하세요"는 "안녕하다"에서 온 말인데, "안녕하다"는 "평화롭다" 또는 "편안하다"는 뜻입니다.  그래서 "안녕하세요"는 "잘 지내시나요?" 또는 "평안하신가요?"와 같은 의미를 지닙니다.\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='현재 한국의 경제 상황에 대해서 설명해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content=' 한국의 경제 상황은 현재 긍정적이지만, 일부 어려움을 겪고 있습니다. \n\n* **긍정적인 측면:**\n    * **수출:** 한국의 수출은 ICT 제품을 중심으로 양호한 흐름을 유지하고 있습니다. 특히 반도체 수출이 높은 증가세를 기록하고 있습니다. \n    * **경제 성장:** 2024년 한국 경제 성장률은 2.2%에서 2.7%로 상향 조정되었습니다. 이는 1분기 경제 성장률이 예상보다 높았고, 수출 경기 회복이 내수 부진을 일부 완화했기 때문입니다.\n    * **설비 투자:** 금리 인하와 반도체 경기 호조세로 설비 투자가 증가할 것으로 예상됩니다. 특히 반도체 관련 투자가 확대되고 있습니다.\n    * **경상수지:** 수출 증가세 둔화와 내수 회복에도 불구하고 교역 조건 개선으로 경상수지 흑자 폭이 확대될 전망입니다. \n    * **신용 등급:** 한국의 신용 등급은 안정적인 수준을 유지하고 있습니다.\n\n* **부정적인 측면:**\n    * **내수 부진:** 건설 투자가 부진하면서 내수 회복이 제약되고 있습니다. \n    * **고용:

In [18]:
# Manual input for chat history if you want to add something usefule to chat history.
chat_history.add_user_message("what's your role?")
chat_history.add_ai_message("I'm a helpful assistant")

chat_history.messages

[HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}),
 AIMessage(content='"안녕하세요"는 한국어에서 가장 일반적인 인사말입니다.  "안녕하세요"는 "무탈하시오?"와 같이 "잘 지내시나요?"라고 해석할 수 있습니다.  "안녕하세요"는 "안녕하다"에서 온 말인데, "안녕하다"는 "평화롭다" 또는 "편안하다"는 뜻입니다.  그래서 "안녕하세요"는 "잘 지내시나요?" 또는 "평안하신가요?"와 같은 의미를 지닙니다.\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='현재 한국의 경제 상황에 대해서 설명해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content=' 한국의 경제 상황은 현재 긍정적이지만, 일부 어려움을 겪고 있습니다. \n\n* **긍정적인 측면:**\n    * **수출:** 한국의 수출은 ICT 제품을 중심으로 양호한 흐름을 유지하고 있습니다. 특히 반도체 수출이 높은 증가세를 기록하고 있습니다. \n    * **경제 성장:** 2024년 한국 경제 성장률은 2.2%에서 2.7%로 상향 조정되었습니다. 이는 1분기 경제 성장률이 예상보다 높았고, 수출 경기 회복이 내수 부진을 일부 완화했기 때문입니다.\n    * **설비 투자:** 금리 인하와 반도체 경기 호조세로 설비 투자가 증가할 것으로 예상됩니다. 특히 반도체 관련 투자가 확대되고 있습니다.\n    * **경상수지:** 수출 증가세 둔화와 내수 회복에도 불구하고 교역 조건 개선으로 경상수지 흑자 폭이 확대될 전망입니다. \n    * **신용 등급:** 한국의 신용 등급은 안정적인 수준을 유지하고 있습니다.\n\n* **부정적인 측면:**\n    * **내수 부진:** 건설 투자가 부진하면서 내수 회복이 제약되고 있습니다. \n    * **고용: